## DST Assessment 3

In this assessment we are going to attempt topic modelling on some cyber security related data.

In [13]:
import pandas as pd
import requests
import datetime as dt
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import numpy as np

In [3]:
start = dt.datetime.now()
df = pd.read_csv("https://github.com/Galeforse/DST-Assessment-03/raw/master/Data/master_log.csv.gz")
print("Data fetched in:" ,dt.datetime.now()-start)

Data fetched in: 0:00:48.150059


In [7]:
print(len(df))
df.head()

15880522


,anon_ip,log
0,161.166.1.23,Jan 5 03:23:40 161.166.1.23 sshd[27076]: pam_...
1,161.166.1.23,Jan 5 03:23:42 161.166.1.23 sshd[27076]: Fail...
2,161.166.1.23,Jan 5 03:23:44 161.166.1.23 sshd[27076]: Fail...
3,161.166.1.23,Jan 5 03:23:47 161.166.1.23 sshd[27076]: Fail...
4,161.166.1.23,Jan 5 03:23:49 161.166.1.23 sshd[27076]: Fail...


In [11]:
data_text = df[['log']]
data_text["index"] = data_text.index
print(len(data_text))
print(data_text[:5])

15880522
                                                 log  index
0  Jan  5 03:23:40 161.166.1.23 sshd[27076]: pam_...      0
1  Jan  5 03:23:42 161.166.1.23 sshd[27076]: Fail...      1
2  Jan  5 03:23:44 161.166.1.23 sshd[27076]: Fail...      2
3  Jan  5 03:23:47 161.166.1.23 sshd[27076]: Fail...      3
4  Jan  5 03:23:49 161.166.1.23 sshd[27076]: Fail...      4


In [17]:
lmtzr = nltk.WordNetLemmatizer().lemmatize

def normalize_text(text):
    ## Runs on documents (vector of words)
    word_pos = nltk.pos_tag(nltk.word_tokenize(text))
    lemm_words = [lmtzr(sw[0], get_wordnet_pos(sw[1])) for sw in word_pos]

    return [x.lower() for x in lemm_words]

def preprocess(text):
    ## Runs on documents (vector of words)
    lemmatizer = nltk.WordNetLemmatizer()
    return([lemmatizer.lemmatize(i) for i in text.split()])

In [24]:
en_stop = set(nltk.corpus.stopwords.words('english'))

from nltk.corpus import wordnet as wn
def get_lemma(word):
    ## morphy does a lemma lookup and word standardization
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

## lemmatize
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

## This version is for comparison
def prepare_text_for_lda(text):
    ## Runs on documents (vector of words)
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [27]:
from gensim import parsing
doc_sample = data_text[data_text['index'] == 16].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print('\n\n simpler tokenized and lemmatized document: ')
print(preprocess(doc_sample))
print('\n\n method removing stop words: ')
print(prepare_text_for_lda(doc_sample))

original document: 


 simpler tokenized and lemmatized document: 
['Jan', '5', '03:24:10', '161.166.1.23', 'sshd[27080]:', 'Failed', 'password', 'for', 'XXXXX', 'from', '5.140.148.58', 'port', '3143', 'ssh2']


 method removing stop words: 
['03:24:10', '161.166.1.23', 'sshd', '27080', 'Failed', 'password', 'XXXXX', '5.140.148.58', 'port', '3143', 'ssh2']


## References

[SSH Topic Modelling](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7117015)